In [12]:
import pandas as pd

# Training OVERLAECOBO
En este notebook se llevaran acabo las pruebas con OVERLAECOBO, se probaran los distintos ZOOMS y la informacion extra para asi escoger la mejor configuracion de dataset y de red neuronal. Como primer paso sera obtener el repositorio donde estan las redes entrenadas y preparalo para su funcionamiento con OVERLAECOBO

In [ ]:
!git clone https://github.com/GeorgeSeif/Semantic-Segmentation-Suite.git

¿Que pruebas se van a realizar?
* Un entrenamiento con cada uno de los zooms sin informacion aumentada
* Un entrenamiento con cada uno de los zooms con informacion aumentada

Las pruebas tendran la etiqueta del lugar donde fueron realizadas, esto debido a que hoy (24 - 05 -18) aun no tengo una cuenta e guane, sin olvidar que cuando la tenga debo hacer configuraciones e instalar tensorflow lo que conllevara bastante tiempo, tiempo en el que podria hacer pruebas en mi pc. Tambien se documentará los parametros a usar, epoch, batch_size, etc.

## 1. Preparar el dataset para usar la Suite
Dado que la suite tiene su propia estructura para el dataset, se hace necesario cambiar los directorios a dicha estructura, y aqui se presenta un inconveniente y es que segun un [issue](https://github.com/GeorgeSeif/Semantic-Segmentation-Suite/issues/65) que encontre hoy (24 - 05 - 18 7:18 am) y segun eso de hace una hora, en el framework no usan las imagenes de test, entonces:
1.  mi test se vuelve el nuevo valid y no existitian imagenes para test y siguo usando el framework
2. cambio todo lo interno del repo para que use test y duro otra semana sacando mi dataset con la nueva division de 3 partes y no solo dos como lo hice
3. busco otro repositorio

Me quedaré con la segunda opcion, la tercera no me cuadra ya con este esta sirviendo, podria atacar el problema con otras cuestiones, primero el issue es muy reciente y podria estar pendiente de cuando haga los arreglos y pues solo seria volver a copiar el repo y si no dan solucion tan solo crear un metodo para sacar un score, para predecir se predice con una imagen, le paso for a la carpeta test, que predicga y luego trato de sacar algun score haciedno comparaciones alfin y alcabo la cuestion es de que entrene

### 1.1 Restructurando dataset
La suite pide una estrucutra de dataset de la siguiente manera.

           - dataset
               - train
               - train_labels
               - valid
               - valid_labels
               - class_dict.scv

cabe aclarar que esta vendria siendo la estructura para un unico zoom asi que dentro de la carpeta actual de OVERLAECOBO creare nuevas carpetas con cada zoom y luego dentro dividir en train y valid.

In [7]:
import os

In [5]:
for i in range(8):
    os.mkdir("./data/dataset/OVERLAECOBO/OVERLAECOBO_"+str(i+13))

In [8]:
for i in range(8):
    os.rename("./data/dataset/OVERLAECOBO/train/satellite/"+str(i+13), "./data/dataset/OVERLAECOBO/OVERLAECOBO_"+str(i+13)+"/train")
    os.rename("./data/dataset/OVERLAECOBO/train/economic_layer/"+str(i+13), "./data/dataset/OVERLAECOBO/OVERLAECOBO_"+str(i+13)+"/train_labels")
    os.rename("./data/dataset/OVERLAECOBO/test/satellite/"+str(i+13), "./data/dataset/OVERLAECOBO/OVERLAECOBO_"+str(i+13)+"/val") 
    os.rename("./data/dataset/OVERLAECOBO/test/economic_layer/"+str(i+13), "./data/dataset/OVERLAECOBO/OVERLAECOBO_"+str(i+13)+"/val_labels")    

Queria copiar con codigo los archivos class_dict pero al parece la libreia os no tiene un `cp` y con shutil habia como 4 metodos para copiar, aisque lo hice manual

### 1.2 Switch Train Test
Dado que al parecer las carpetas valid tienes mas datos de las de train hare un switch entoncs valid pasara a ser train y train valid

In [11]:
for i in range(7):
    os.rename("./OVERLAECOBO_"+str(i+14)+"/train","./OVERLAECOBO_"+str(i+14)+"/pibote")
    os.rename("./OVERLAECOBO_"+str(i+14)+"/train_labels","./OVERLAECOBO_"+str(i+14)+"/pibote_labels")
    os.rename("./OVERLAECOBO_"+str(i+14)+"/valid","./OVERLAECOBO_"+str(i+14)+"/train")
    os.rename("./OVERLAECOBO_"+str(i+14)+"/valid_labels","./OVERLAECOBO_"+str(i+14)+"/train_labels")
    os.rename("./OVERLAECOBO_"+str(i+14)+"/pibote","./OVERLAECOBO_"+str(i+14)+"/val")
    os.rename("./OVERLAECOBO_"+str(i+14)+"/pibote_labels","./OVERLAECOBO_"+str(i+14)+"/val_labels")   

## 2. Metricas
Al parecer el framework usa varias metricas provenientes de sklearn, y lo se dado a que en uno de sus archivos `.py` se encuentra el siguiente import

      from sklearn.metrics import precision_score, recall_score, confusion_matrix, classification_report, accuracy_score, f1_score
   
y mirando en la documentacion de sklearn, dichas metricas significan [precision_score](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.precision_score.html#sklearn.metrics.precision_score), [recall_score](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.recall_score.html#sklearn.metrics.recall_score), [f1_score](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html#sklearn.metrics.f1_score)

## 3. Pruebas
Y por ahora (24 - May - 2018) que aun no tengo la cuenta de guane iniciare las pruebas e mi GPU y con los ZOOMs mas grandes puesto que hay menos imagenes. una de las cosas es que podria probar varios modelos, entre los que permite la suite

* DenseNet56
* FC-DenseNet67
* FC-DenseNet103 
* Encoder-Decoder
* Encoder-Decoder-Skip
* RefineNet-Res50
* RefineNet-Res101 
* RefineNet-Res152
* FRRN-A
* FRRN-B
* MobileUNet
* MobileUNet-Skip
* PSPNet-Res50 
* PSPNet-Res101
* PSPNet-Res152
* GCN-Res50 
* GCN-Res101
* GCN-Res152

Para cada prueba se realizara un train, un test y un predcit. el predict será de una imagen cualquiera del directorio valid

### 3.1 Sin Información extra

Las pruebas a realizar a continuacion son pruebas echas meramente con los 3 canales de una imagen RGB, para las pruebas con informacion extra se agregaran uno o mas canales, dependiendo de la informacion obtenida y la capacidad del framework

<h4 style="color:#c62828"> Prueba # 1 </h4>

Prueba realizada con los siguientes parametros

In [2]:
DATASET_PATH = "OVERLAECOBO_13"
CROP_SIZE = 256
NUM_EPOCH = 20
BATCH_SIZE = 1 # Hay pocas imagenes por ende se prueba con un batch pequeño
MODEL = "FC-DenseNet56"

In [ ]:
%%time
!python main.py --mode train --dataset {DATASET_PATH} --crop_height {CROP_SIZE} --crop_width {CROP_SIZE} --num_epochs {NUM_EPOCH} --batch_size {BATCH_SIZE} --model {MODEL}

#### Resultados:
##### Train results
<center>
<div style="width:60%">
    
![accuracy_vs_epochs](./pruebas/prueba 1/accuracy_vs_epochs.png)
</div>
</center>

<center>
<div style="width:60%">
    
![loss_vs_epochs](./pruebas/prueba 1/loss_vs_epochs.png)
</div>
</center>

##### Test Result

In [13]:
test_csv = pd.read_csv("./pruebas/prueba 1/val_scores.csv")
test_csv

,val_name,avg_accuracy,precision,recall,f1 score,mean iou Estrato_1,Estrato_2,Estrato_3,Estrato_4,Estrato_5,Estrato_6,No_Definido
0,0.077682,0.567242,0.077682,0.070037,0.025534,0.047451,0.038488,0.031355,0.018237,1.0,1.0,0.748466
1,0.304291,0.564342,0.304291,0.363517,0.105886,0.066800,0.043206,0.042991,1.000000,1.0,1.0,0.789856


<h4 style="color:#c62828"> Prueba # 2</h4>

In [ ]:
DATASET_PATH = "OVERLAECOBO_13"
CROP_SIZE = 256
NUM_EPOCH = 30
BATCH_SIZE = 1 # Hay pocas imagenes por ende se prueba con un batch pequeño y al usar 2 sale un error que no se que sera pero creo que es algo con la GPU
MODEL = "FC-DenseNet56"

#### Resultados:
##### Train results
<center>
<div style="width:60%">
    
![accuracy_vs_epochs](./pruebas/prueba 2/accuracy_vs_epochs.png)
</div>
</center>

<center>
<div style="width:60%">
    
![loss_vs_epochs](./pruebas/prueba 2/loss_vs_epochs.png)
</div>
</center>

##### Test Result

In [16]:
test_csv = pd.read_csv("./pruebas/prueba 2/val_scores.csv")
test_csv

,val_name,avg_accuracy,precision,recall,f1 score,mean iou Estrato_1,Estrato_2,Estrato_3,Estrato_4,Estrato_5,Estrato_6,No_Definido
0,0.071991,0.507403,0.071991,0.066388,0.024978,0.042428,0.046735,0.045053,0.018237,1.0,1.0,0.700409
1,0.295135,0.556844,0.295135,0.359648,0.103411,0.052387,0.060627,0.039252,1.000000,1.0,1.0,0.789023


<h4 style="color:#c62828"> Prueba # 3 </h4>

In [ ]:
DATASET_PATH = "OVERLAECOBO_13"
CROP_SIZE = 256
NUM_EPOCH = 50
BATCH_SIZE = 1 # Hay pocas imagenes por ende se prueba con un batch pequeño y al usar 2 sale un error que no se que sera pero creo que es algo con la GPU
MODEL = "FC-DenseNet56"

#### Resultados:
##### Train results
<center>
<div style="width:60%">
    
![accuracy_vs_epochs](./pruebas/prueba 3/accuracy_vs_epochs.png)
</div>
</center>

<center>
<div style="width:60%">
    
![loss_vs_epochs](./pruebas/prueba 3/loss_vs_epochs.png)
</div>
</center>

##### Test Result

In [19]:
test_csv = pd.read_csv("./pruebas/prueba 3/val_scores.csv")
test_csv

,val_name,avg_accuracy,precision,recall,f1 score,mean iou Estrato_1,Estrato_2,Estrato_3,Estrato_4,Estrato_5,Estrato_6,No_Definido
0,0.120483,0.521630,0.120483,0.086283,0.038158,0.092034,0.043299,0.093455,0.021277,1.0,1.0,0.758010
1,0.333466,0.567112,0.333466,0.376041,0.120136,0.101820,0.041812,0.104673,1.000000,1.0,1.0,0.808893


<h4 style="color:#c62828"> Prueba # 4 </h4>

In [ ]:
DATASET_PATH = "OVERLAECOBO_13"
CROP_SIZE = 256
NUM_EPOCH = 50
BATCH_SIZE = 1 # Hay pocas imagenes por ende se prueba con un batch pequeño y al usar 2 sale un error que no se que sera pero creo que es algo con la GPU
MODEL = "FC-DenseNet56"

#### Resultados:
##### Train results
<center>
<div style="width:60%">
    
![accuracy_vs_epochs](./pruebas/prueba 4/accuracy_vs_epochs.png)
</div>
</center>

<center>
<div style="width:60%">
    
![loss_vs_epochs](./pruebas/prueba 4/loss_vs_epochs.png)
</div>
</center>

##### Test Result

In [22]:
test_csv = pd.read_csv("./pruebas/prueba 4/val_scores.csv")
test_csv

,val_name,avg_accuracy,precision,recall,f1 score,mean iou Estrato_1,Estrato_2,Estrato_3,Estrato_4,Estrato_5,Estrato_6,No_Definido
0,0.065903,0.526904,0.065903,0.075991,0.029102,0.027462,0.118213,0.123592,0.033435,1.0,1.0,0.732788
1,0.281158,0.576085,0.281158,0.358391,0.100384,0.031137,0.098955,0.209346,1.000000,1.0,1.0,0.781149


<h4 style="color:#c62828"> Prueba # 5 </h4>

In [ ]:
DATASET_PATH = "OVERLAECOBO_13"
CROP_SIZE = 256
NUM_EPOCH = 50
BATCH_SIZE = 1 # Hay pocas imagenes por ende se prueba con un batch pequeño y al usar 2 sale un error que no se que sera pero creo que es algo con la GPU
MODEL = "Encoder-Decoder"

#### Resultados:
##### Train results
<center>
<div style="width:60%">
    
![accuracy_vs_epochs](./pruebas/prueba 5/accuracy_vs_epochs.png)
</div>
</center>

<center>
<div style="width:60%">
    
![loss_vs_epochs](./pruebas/prueba 5/loss_vs_epochs.png)
</div>
</center>

##### Test Result

In [26]:
test_csv = pd.read_csv("./pruebas/prueba 5/val_scores.csv")
test_csv

FileNotFoundError: File b'./pruebas/prueba 5/val_scores.csv' does not exist

<h4 style="color:#c62828"> Prueba # 6 </h4>

In [ ]:
DATASET_PATH = "OVERLAECOBO_13"
CROP_SIZE = 256
NUM_EPOCH = 50
BATCH_SIZE = 2 # Hay pocas imagenes por ende se prueba con un batch pequeño y al usar 2 sale un error que no se que sera pero creo que es algo con la GPU
MODEL = "Encoder-Decoder"

#### Resultados:
##### Train results
<center>
<div style="width:60%">
    
![accuracy_vs_epochs](./pruebas/prueba 5/accuracy_vs_epochs.png)
</div>
</center>

<center>
<div style="width:60%">
    
![loss_vs_epochs](./pruebas/prueba 5/loss_vs_epochs.png)
</div>
</center>

##### Test Result

In [28]:
test_csv = pd.read_csv("./pruebas/prueba 6/val_scores.csv")
test_csv

,val_name,avg_accuracy,precision,recall,f1 score,mean iou Estrato_1,Estrato_2,Estrato_3,Estrato_4,Estrato_5,Estrato_6,No_Definido
0,0.070969,0.684431,0.070969,0.090032,0.024175,0.026889,0.617182,0.0,0.0,1.0,1.0,0.751875
1,0.317337,0.767017,0.317337,0.395334,0.122857,0.038451,0.719861,0.0,1.0,1.0,1.0,0.840667


## Prueba multi capa

In [2]:
import os
from skimage import io
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
%matplotlib inline

In [9]:
for i in range(7):
    img = io.imread("./OVERLAECOBO_13_multi/train/"+str(i)+".png")
    d = img.flatten()
    f = list(d) + list(d)
    g = np.array(f).reshape((256,256,6))[:,:,:4]
    io.imsave("./OVERLAECOBO_13_multi/train/"+str(i)+".png",g)

PDTA: Funciona!!!!!!!!

<h4 style="color:#c62828"> Prueba # 7 </h4>

In [ ]:
DATASET_PATH = "OVERLAECOBO_13"
CROP_SIZE = 256
NUM_EPOCH = 50
BATCH_SIZE = 1 # Hay pocas imagenes por ende se prueba con un batch pequeño y al usar 2 sale un error que no se que sera pero creo que es algo con la GPU
MODEL = "Encoder-Decoder-Skip"

#### Resultados:
##### Train results
<center>
<div style="width:60%">
    
![accuracy_vs_epochs](./pruebas/prueba 7/accuracy_vs_epochs.png)
</div>
</center>

<center>
<div style="width:60%">
    
![loss_vs_epochs](./pruebas/prueba 7/loss_vs_epochs.png)
</div>
</center>

##### Test Result

In [30]:
test_csv = pd.read_csv("./pruebas/prueba 7/val_scores.csv")
test_csv

,val_name,avg_accuracy,precision,recall,f1 score,mean iou Estrato_1,Estrato_2,Estrato_3,Estrato_4,Estrato_5,Estrato_6,No_Definido
0,0.051758,0.663817,0.051758,0.088217,0.024394,0.004415,0.358763,0.008828,0.243161,1.0,1.0,0.854465
1,0.294052,0.612204,0.294052,0.383504,0.119764,0.002382,0.411150,0.003738,1.000000,1.0,1.0,0.860537
